# 🧪 Phase-3.3 Validation Tests
## Evidence Accumulation Testing

---

### 🎯 **Test Coverage**

This notebook validates:
1. **Basic Accumulation:** Correct grouping and counting
2. **Similarity Aggregation:** Mean, max, min calculations
3. **Recurrence Scoring:** Fraction of flows containing attack
4. **Threat Ranking:** Repeated moderate > single strong
5. **Threshold Filtering:** min_similarity_threshold handling
6. **Edge Cases:** Empty window, no results, single flow

### ✅ **Success Criteria**
- Correct attack type grouping
- Accurate metric computation
- Proper threat score ranking
- Edge cases handled gracefully

---

## 📦 Setup and Imports

In [1]:
import sys
import numpy as np
from typing import List
import matplotlib.pyplot as plt

# Import Phase-3.3 functions
%run Phase_3_3_Evidence_Accumulation.ipynb

print("✅ Setup complete")
print(f"   FlowRecord: {FlowRecord}")
print(f"   AttackEvidence: {AttackEvidence}")
print(f"   accumulate_evidence: {accumulate_evidence}")

✅ Libraries imported successfully
✅ FlowRecord dataclass defined
✅ AdaptiveTimeWindow class defined
Phase-3.1: Adaptive Time Window Implementation
✅ AdaptiveTimeWindow initialized
   Min window: 5.0s / 10 flows
   Max window: 30.0s / 100 flows
   Initial state: AdaptiveTimeWindow(flows=0, span=0.0s, state=empty)
✅ Phase-3.1 class implementation complete!
   Run validation notebook for comprehensive tests.
✅ Libraries imported successfully
ChromaDB Collection Loaded
Collection: iot_behavioral_memory_curated
Total Vectors: 17,712
Metadata: {'distance_metric': 'cosine', 'created_at': '2026-02-04 22:14:31', 'dimensions': 99, 'description': 'Curated behavioral patterns for Quantum-RAG IDS', 'attack_samples': 4428, 'benign_samples': 13284, 'ingestion_strategy': 'curated_representative', 'total_samples': 17712}
✅ ChromaDB client ready for retrieval
✅ retrieve_behavioral_evidence() function defined
Testing retrieval with synthetic vector...
Vector shape: (99,)

RETRIEVAL TEST RESULTS
Latency: 

---

## 🧪 Test 1: Basic Accumulation

Verify correct grouping and counting of matches.

In [2]:
print("="*80)
print("TEST 1: Basic Accumulation")
print("="*80)

# Create test flows with known retrieval results
test_flows = []

# Flow 1: 2 backdoor + 1 ddos
flow1 = FlowRecord(
    flow_id="test_001",
    timestamp=100.0,
    vector_embedding=np.random.rand(99).astype(np.float32),
    retrieval_results=[
        {'similarity': 0.8, 'distance': 0.25, 'attack_type': 'backdoor', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.7, 'distance': 0.43, 'attack_type': 'backdoor', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.9, 'distance': 0.11, 'attack_type': 'ddos', 'label': 'Attack', 'metadata': {}},
    ],
    metadata={}
)

# Flow 2: 1 backdoor + 2 ransomware
flow2 = FlowRecord(
    flow_id="test_002",
    timestamp=105.0,
    vector_embedding=np.random.rand(99).astype(np.float32),
    retrieval_results=[
        {'similarity': 0.75, 'distance': 0.33, 'attack_type': 'backdoor', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.85, 'distance': 0.18, 'attack_type': 'ransomware', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.82, 'distance': 0.22, 'attack_type': 'ransomware', 'label': 'Attack', 'metadata': {}},
    ],
    metadata={}
)

# Flow 3: 1 ddos
flow3 = FlowRecord(
    flow_id="test_003",
    timestamp=110.0,
    vector_embedding=np.random.rand(99).astype(np.float32),
    retrieval_results=[
        {'similarity': 0.88, 'distance': 0.14, 'attack_type': 'ddos', 'label': 'Attack', 'metadata': {}},
    ],
    metadata={}
)

test_flows = [flow1, flow2, flow3]

# Accumulate
evidence = accumulate_evidence(test_flows)

print(f"✓ Accumulated evidence from {len(test_flows)} flows")
print(f"✓ Detected {len(evidence)} attack types")

# Validate counts
assert 'backdoor' in evidence, "Missing 'backdoor' attack type"
assert 'ddos' in evidence, "Missing 'ddos' attack type"
assert 'ransomware' in evidence, "Missing 'ransomware' attack type"

backdoor = evidence['backdoor']
ddos = evidence['ddos']
ransomware = evidence['ransomware']

# Expected counts: backdoor=3, ddos=2, ransomware=2
assert backdoor.count == 3, f"Expected backdoor count=3, got {backdoor.count}"
assert ddos.count == 2, f"Expected ddos count=2, got {ddos.count}"
assert ransomware.count == 2, f"Expected ransomware count=2, got {ransomware.count}"

print(f"✓ Correct counts: backdoor={backdoor.count}, ddos={ddos.count}, ransomware={ransomware.count}")

# Validate attack types
assert backdoor.attack_type == 'backdoor'
assert ddos.attack_type == 'ddos'
assert ransomware.attack_type == 'ransomware'

print(f"✓ Attack types correctly assigned")

print("\n✅ TEST 1 PASSED: Basic accumulation correct")

TEST 1: Basic Accumulation
✓ Accumulated evidence from 3 flows
✓ Detected 3 attack types
✓ Correct counts: backdoor=3, ddos=2, ransomware=2
✓ Attack types correctly assigned

✅ TEST 1 PASSED: Basic accumulation correct


## 🧪 Test 2: Similarity Aggregation

Verify mean, max, min similarity calculations.

In [3]:
print("="*80)
print("TEST 2: Similarity Aggregation")
print("="*80)

# Use same test flows from Test 1
# backdoor similarities: [0.8, 0.7, 0.75]
# ddos similarities: [0.9, 0.88]
# ransomware similarities: [0.85, 0.82]

backdoor = evidence['backdoor']
ddos = evidence['ddos']
ransomware = evidence['ransomware']

# Validate backdoor
expected_backdoor_avg = np.mean([0.8, 0.7, 0.75])
expected_backdoor_max = 0.8
expected_backdoor_min = 0.7

assert abs(backdoor.avg_similarity - expected_backdoor_avg) < 1e-6, \
    f"Backdoor avg_similarity incorrect: {backdoor.avg_similarity} vs {expected_backdoor_avg}"
assert abs(backdoor.max_similarity - expected_backdoor_max) < 1e-6, \
    f"Backdoor max_similarity incorrect"
assert abs(backdoor.min_similarity - expected_backdoor_min) < 1e-6, \
    f"Backdoor min_similarity incorrect"

print(f"✓ Backdoor: avg={backdoor.avg_similarity:.4f}, max={backdoor.max_similarity:.4f}, min={backdoor.min_similarity:.4f}")

# Validate ddos
expected_ddos_avg = np.mean([0.9, 0.88])
expected_ddos_max = 0.9
expected_ddos_min = 0.88

assert abs(ddos.avg_similarity - expected_ddos_avg) < 1e-6, \
    f"DDoS avg_similarity incorrect"
assert abs(ddos.max_similarity - expected_ddos_max) < 1e-6, \
    f"DDoS max_similarity incorrect"
assert abs(ddos.min_similarity - expected_ddos_min) < 1e-6, \
    f"DDoS min_similarity incorrect"

print(f"✓ DDoS: avg={ddos.avg_similarity:.4f}, max={ddos.max_similarity:.4f}, min={ddos.min_similarity:.4f}")

# Validate ransomware
expected_ransomware_avg = np.mean([0.85, 0.82])
expected_ransomware_max = 0.85
expected_ransomware_min = 0.82

assert abs(ransomware.avg_similarity - expected_ransomware_avg) < 1e-6, \
    f"Ransomware avg_similarity incorrect"
assert abs(ransomware.max_similarity - expected_ransomware_max) < 1e-6, \
    f"Ransomware max_similarity incorrect"
assert abs(ransomware.min_similarity - expected_ransomware_min) < 1e-6, \
    f"Ransomware min_similarity incorrect"

print(f"✓ Ransomware: avg={ransomware.avg_similarity:.4f}, max={ransomware.max_similarity:.4f}, min={ransomware.min_similarity:.4f}")

print("\n✅ TEST 2 PASSED: Similarity aggregation accurate")

TEST 2: Similarity Aggregation
✓ Backdoor: avg=0.7500, max=0.8000, min=0.7000
✓ DDoS: avg=0.8900, max=0.9000, min=0.8800
✓ Ransomware: avg=0.8350, max=0.8500, min=0.8200

✅ TEST 2 PASSED: Similarity aggregation accurate


## 🧪 Test 3: Recurrence Scoring

Verify recurrence = fraction of flows containing each attack type.

In [4]:
print("="*80)
print("TEST 3: Recurrence Scoring")
print("="*80)

# From Test 1:
# Total flows: 3
# backdoor: appears in flows [test_001, test_002] → 2/3 = 0.667
# ddos: appears in flows [test_001, test_003] → 2/3 = 0.667
# ransomware: appears in flows [test_002] → 1/3 = 0.333

expected_backdoor_recurrence = 2 / 3
expected_ddos_recurrence = 2 / 3
expected_ransomware_recurrence = 1 / 3

assert abs(backdoor.recurrence_score - expected_backdoor_recurrence) < 1e-6, \
    f"Backdoor recurrence incorrect: {backdoor.recurrence_score} vs {expected_backdoor_recurrence}"
assert abs(ddos.recurrence_score - expected_ddos_recurrence) < 1e-6, \
    f"DDoS recurrence incorrect"
assert abs(ransomware.recurrence_score - expected_ransomware_recurrence) < 1e-6, \
    f"Ransomware recurrence incorrect"

print(f"✓ Backdoor recurrence: {backdoor.recurrence_score:.4f} (appears in {len(backdoor.flow_ids)}/3 flows)")
print(f"✓ DDoS recurrence: {ddos.recurrence_score:.4f} (appears in {len(ddos.flow_ids)}/3 flows)")
print(f"✓ Ransomware recurrence: {ransomware.recurrence_score:.4f} (appears in {len(ransomware.flow_ids)}/3 flows)")

# Validate flow_ids
assert set(backdoor.flow_ids) == {'test_001', 'test_002'}, "Backdoor flow_ids incorrect"
assert set(ddos.flow_ids) == {'test_001', 'test_003'}, "DDoS flow_ids incorrect"
assert set(ransomware.flow_ids) == {'test_002'}, "Ransomware flow_ids incorrect"

print(f"✓ Flow IDs correctly tracked")

print("\n✅ TEST 3 PASSED: Recurrence scoring accurate")

TEST 3: Recurrence Scoring
✓ Backdoor recurrence: 0.6667 (appears in 2/3 flows)
✓ DDoS recurrence: 0.6667 (appears in 2/3 flows)
✓ Ransomware recurrence: 0.3333 (appears in 1/3 flows)
✓ Flow IDs correctly tracked

✅ TEST 3 PASSED: Recurrence scoring accurate


## 🧪 Test 4: Threat Ranking

Verify that repeated moderate similarity beats single strong match.

In [5]:
print("="*80)
print("TEST 4: Threat Ranking (Repeated Moderate > Single Strong)")
print("="*80)

# Create scenario:
# Attack A: High recurrence (0.8), moderate similarity (0.6)
# Attack B: Low recurrence (0.2), high similarity (0.9)

ranking_flows = []

# 8 flows with Attack A (moderate similarity)
for i in range(8):
    flow = FlowRecord(
        flow_id=f"rank_flow_{i:03d}",
        timestamp=200.0 + i * 5.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[
            {'similarity': 0.6, 'distance': 0.67, 'attack_type': 'attack_a', 'label': 'Attack', 'metadata': {}},
        ],
        metadata={}
    )
    ranking_flows.append(flow)

# 2 flows with Attack B (high similarity)
for i in range(8, 10):
    flow = FlowRecord(
        flow_id=f"rank_flow_{i:03d}",
        timestamp=200.0 + i * 5.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[
            {'similarity': 0.9, 'distance': 0.11, 'attack_type': 'attack_b', 'label': 'Attack', 'metadata': {}},
        ],
        metadata={}
    )
    ranking_flows.append(flow)

# Accumulate
ranking_evidence = accumulate_evidence(ranking_flows)

attack_a = ranking_evidence['attack_a']
attack_b = ranking_evidence['attack_b']

print(f"\nAttack A:")
print(f"  Count: {attack_a.count}")
print(f"  Recurrence: {attack_a.recurrence_score:.2f} (8/10 flows)")
print(f"  Avg Similarity: {attack_a.avg_similarity:.4f}")

print(f"\nAttack B:")
print(f"  Count: {attack_b.count}")
print(f"  Recurrence: {attack_b.recurrence_score:.2f} (2/10 flows)")
print(f"  Avg Similarity: {attack_b.avg_similarity:.4f}")

# Calculate threat scores (60% recurrence, 40% similarity)
threat_a = 0.6 * attack_a.recurrence_score + 0.4 * attack_a.avg_similarity
threat_b = 0.6 * attack_b.recurrence_score + 0.4 * attack_b.avg_similarity

print(f"\nThreat Scores:")
print(f"  Attack A: {threat_a:.4f}")
print(f"  Attack B: {threat_b:.4f}")

# Rank
ranked = rank_evidence_by_threat(ranking_evidence)

print(f"\nRanked Order:")
for i, ev in enumerate(ranked, 1):
    threat_score = 0.6 * ev.recurrence_score + 0.4 * ev.avg_similarity
    print(f"  {i}. {ev.attack_type} (threat={threat_score:.4f})")

# Validate: Attack A should rank higher
assert ranked[0].attack_type == 'attack_a', "Attack A should rank #1 (repeated moderate)"
assert ranked[1].attack_type == 'attack_b', "Attack B should rank #2 (single strong)"
assert threat_a > threat_b, "Attack A threat score should exceed Attack B"

print(f"\n✓ Validation: Attack A (repeated moderate) ranks higher than Attack B (single strong)")

print("\n✅ TEST 4 PASSED: Threat ranking correct")

TEST 4: Threat Ranking (Repeated Moderate > Single Strong)

Attack A:
  Count: 8
  Recurrence: 0.80 (8/10 flows)
  Avg Similarity: 0.6000

Attack B:
  Count: 2
  Recurrence: 0.20 (2/10 flows)
  Avg Similarity: 0.9000

Threat Scores:
  Attack A: 0.7200
  Attack B: 0.4800

Ranked Order:
  1. attack_a (threat=0.7200)
  2. attack_b (threat=0.4800)

✓ Validation: Attack A (repeated moderate) ranks higher than Attack B (single strong)

✅ TEST 4 PASSED: Threat ranking correct


## 🧪 Test 5: Threshold Filtering

Verify min_similarity_threshold filters low-confidence matches.

In [6]:
print("="*80)
print("TEST 5: Threshold Filtering")
print("="*80)

# Create flows with varying similarities
threshold_flows = []

flow = FlowRecord(
    flow_id="thresh_001",
    timestamp=300.0,
    vector_embedding=np.random.rand(99).astype(np.float32),
    retrieval_results=[
        {'similarity': 0.9, 'distance': 0.11, 'attack_type': 'high_conf', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.6, 'distance': 0.67, 'attack_type': 'medium_conf', 'label': 'Attack', 'metadata': {}},
        {'similarity': 0.3, 'distance': 2.33, 'attack_type': 'low_conf', 'label': 'Attack', 'metadata': {}},
    ],
    metadata={}
)

threshold_flows.append(flow)

# Test 1: No threshold (include all)
evidence_no_thresh = accumulate_evidence(threshold_flows, min_similarity_threshold=0.0)
assert len(evidence_no_thresh) == 3, "Should have 3 attack types with threshold=0.0"
assert 'high_conf' in evidence_no_thresh
assert 'medium_conf' in evidence_no_thresh
assert 'low_conf' in evidence_no_thresh
print(f"✓ Threshold=0.0: {len(evidence_no_thresh)} attack types (all included)")

# Test 2: Threshold=0.5 (exclude low_conf)
evidence_thresh_05 = accumulate_evidence(threshold_flows, min_similarity_threshold=0.5)
assert len(evidence_thresh_05) == 2, "Should have 2 attack types with threshold=0.5"
assert 'high_conf' in evidence_thresh_05
assert 'medium_conf' in evidence_thresh_05
assert 'low_conf' not in evidence_thresh_05, "low_conf should be filtered"
print(f"✓ Threshold=0.5: {len(evidence_thresh_05)} attack types (low_conf filtered)")

# Test 3: Threshold=0.8 (only high_conf)
evidence_thresh_08 = accumulate_evidence(threshold_flows, min_similarity_threshold=0.8)
assert len(evidence_thresh_08) == 1, "Should have 1 attack type with threshold=0.8"
assert 'high_conf' in evidence_thresh_08
assert 'medium_conf' not in evidence_thresh_08, "medium_conf should be filtered"
assert 'low_conf' not in evidence_thresh_08, "low_conf should be filtered"
print(f"✓ Threshold=0.8: {len(evidence_thresh_08)} attack type (only high_conf)")

# Test 4: Threshold=1.0 (exclude all)
evidence_thresh_10 = accumulate_evidence(threshold_flows, min_similarity_threshold=1.0)
assert len(evidence_thresh_10) == 0, "Should have 0 attack types with threshold=1.0"
print(f"✓ Threshold=1.0: {len(evidence_thresh_10)} attack types (all filtered)")

print("\n✅ TEST 5 PASSED: Threshold filtering works correctly")

TEST 5: Threshold Filtering
✓ Threshold=0.0: 3 attack types (all included)
✓ Threshold=0.5: 2 attack types (low_conf filtered)
✓ Threshold=0.8: 1 attack type (only high_conf)
✓ Threshold=1.0: 0 attack types (all filtered)

✅ TEST 5 PASSED: Threshold filtering works correctly


## 🧪 Test 6: Edge Cases

Handle empty window, no retrieval results, single flow.

In [7]:
print("="*80)
print("TEST 6: Edge Cases")
print("="*80)

# Case A: Empty window
print("\nCase A: Empty window...")
empty_flows = []
empty_evidence = accumulate_evidence(empty_flows)
assert len(empty_evidence) == 0, "Empty window should return empty evidence"
print(f"  ✓ Empty window handled: {len(empty_evidence)} attack types")

# Case B: Flows with no retrieval_results
print("\nCase B: Flows with no retrieval_results...")
no_results_flows = [
    FlowRecord(
        flow_id="no_result_001",
        timestamp=400.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=None,  # No retrieval
        metadata={}
    ),
    FlowRecord(
        flow_id="no_result_002",
        timestamp=405.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[],  # Empty list
        metadata={}
    )
]
no_results_evidence = accumulate_evidence(no_results_flows)
assert len(no_results_evidence) == 0, "Flows without results should return empty evidence"
print(f"  ✓ No retrieval results handled: {len(no_results_evidence)} attack types")

# Case C: Single flow
print("\nCase C: Single flow...")
single_flow = [
    FlowRecord(
        flow_id="single_001",
        timestamp=500.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[
            {'similarity': 0.8, 'distance': 0.25, 'attack_type': 'solo_attack', 'label': 'Attack', 'metadata': {}},
        ],
        metadata={}
    )
]
single_evidence = accumulate_evidence(single_flow)
assert len(single_evidence) == 1, "Single flow should accumulate correctly"
assert 'solo_attack' in single_evidence
assert single_evidence['solo_attack'].count == 1
assert single_evidence['solo_attack'].recurrence_score == 1.0, "Single flow recurrence should be 1.0"
print(f"  ✓ Single flow handled: {len(single_evidence)} attack type, recurrence=1.0")

# Case D: Mixed - some flows with results, some without
print("\nCase D: Mixed flows...")
mixed_flows = [
    FlowRecord(
        flow_id="mixed_001",
        timestamp=600.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[
            {'similarity': 0.7, 'distance': 0.43, 'attack_type': 'mixed_attack', 'label': 'Attack', 'metadata': {}},
        ],
        metadata={}
    ),
    FlowRecord(
        flow_id="mixed_002",
        timestamp=605.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=None,  # No results
        metadata={}
    ),
    FlowRecord(
        flow_id="mixed_003",
        timestamp=610.0,
        vector_embedding=np.random.rand(99).astype(np.float32),
        retrieval_results=[
            {'similarity': 0.8, 'distance': 0.25, 'attack_type': 'mixed_attack', 'label': 'Attack', 'metadata': {}},
        ],
        metadata={}
    )
]
mixed_evidence = accumulate_evidence(mixed_flows)
assert len(mixed_evidence) == 1
assert mixed_evidence['mixed_attack'].count == 2
assert mixed_evidence['mixed_attack'].recurrence_score == 2/3, "Recurrence should be 2/3"
print(f"  ✓ Mixed flows handled: 2 matches from 3 flows, recurrence=0.67")

print("\n✅ TEST 6 PASSED: All edge cases handled correctly")

TEST 6: Edge Cases

Case A: Empty window...
  ✓ Empty window handled: 0 attack types

Case B: Flows with no retrieval_results...
  ✓ No retrieval results handled: 0 attack types

Case C: Single flow...
  ✓ Single flow handled: 1 attack type, recurrence=1.0

Case D: Mixed flows...
  ✓ Mixed flows handled: 2 matches from 3 flows, recurrence=0.67

✅ TEST 6 PASSED: All edge cases handled correctly


---

## 📊 Final Test Summary

In [8]:
print("\n" + "="*80)
print("TEST SUITE SUMMARY")
print("="*80)
print("Total tests: 6")
print("✅ Passed: 6")
print("❌ Failed: 0")
print("\n🎉 ALL TESTS PASSED! Phase-3.3 validation complete.")
print("\nKEY VALIDATIONS:")
print("  ✅ Attack type grouping correct")
print("  ✅ Similarity aggregation accurate (mean, max, min)")
print("  ✅ Recurrence scoring correct (fraction of flows)")
print("  ✅ Threat ranking validates 'repeated moderate > single strong'")
print("  ✅ Threshold filtering works correctly")
print("  ✅ Edge cases handled (empty, no results, single flow)")
print("\n✅ Evidence accumulation function ready for production use.")
print("\n→ Proceed to Phase-3.4: Quantum-Inspired Fusion")
print("="*80)


TEST SUITE SUMMARY
Total tests: 6
✅ Passed: 6
❌ Failed: 0

🎉 ALL TESTS PASSED! Phase-3.3 validation complete.

KEY VALIDATIONS:
  ✅ Attack type grouping correct
  ✅ Similarity aggregation accurate (mean, max, min)
  ✅ Recurrence scoring correct (fraction of flows)
  ✅ Threat ranking validates 'repeated moderate > single strong'
  ✅ Threshold filtering works correctly
  ✅ Edge cases handled (empty, no results, single flow)

✅ Evidence accumulation function ready for production use.

→ Proceed to Phase-3.4: Quantum-Inspired Fusion


---

## 🎉 Phase-3.3 Validation Complete!

### ✅ All Tests Passed
- Basic accumulation ✅
- Similarity aggregation ✅
- Recurrence scoring ✅
- Threat ranking ✅
- Threshold filtering ✅
- Edge cases ✅

### 📊 Key Insights
- **Grouping:** Matches correctly grouped by attack_type
- **Metrics:** Accurate computation of count, avg/max/min similarity, recurrence
- **Ranking:** Validated principle: repeated moderate (recurrence=0.8, sim=0.6) beats single strong (recurrence=0.2, sim=0.9)
- **Robustness:** Handles empty windows, missing results, single flows

### 🚀 Ready for Phase-3.4
Quantum-inspired fusion to convert evidence → threat probabilities

---